# debug Class

> API details.

In [6]:
#hide
import logging
logging.basicConfig(level= logging.WARNING)
log = logging.getLogger("pynamodb")
log.setLevel(logging.DEBUG)
log.setLevel(logging.WARNING)
log.propagate = True


In [7]:
#hide
import pickle, os, json

os.environ['DATABASE_TABLE_NAME'] = 'product-table-dev-manual'
os.environ['BRANCH'] = 'dev'
os.environ['REGION'] = 'ap-southeast-1'
os.environ['INVENTORY_BUCKET_NAME'] = 'product-bucket-dev-manual'
os.environ['INPUT_BUCKET_NAME'] = 'input-product-bucket-dev-manual'
os.environ['DAX_ENDPOINT'] = 'longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111'
os.environ['LINEKEY'] = '2uAfV4AoYglUGmKTAk2xNOm0aV2Ufgh1BQPvQl9vJd4'
os.environ['INTCOLS'] = json.dumps(['cprcode', 'iprcode', 'oprcode',  'sellingPrice'])
REGION = 'ap-southeast-1'

In [29]:

BRANCH='dev'
ECOMMERCE_COL_LIST = f'https://raw.githubusercontent.com/thanakijwanavit/villaMasterSchema/{BRANCH}/product/ecommerceColList.yaml'

In [8]:
#hide
print(json.dumps(os.environ['INTCOLS']))

"[\"cprcode\", \"iprcode\", \"oprcode\", \"sellingPrice\"]"


In [30]:
from villaProductDatabase.database import ProductDatabase, lambdaDumpOnlineS3
import pandas as pd
from typing import List
import yaml, requests

## LambdaDumpOnline

In [12]:
lambdaDumpOnlineS3({})

ecommece col list is https://raw.githubusercontent.com/thanakijwanavit/villaMasterSchema/dev/product/ecommerceColList.yaml


/home/ec2-user/SageMaker/stacks/villaMaster/villa-master-dev/product/database2/villaProductDatabase/database.py:147: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  onlineList:List[str] = yaml.load(requests.get(ECOMMERCE_COL_LIST).content)


shape is: (20158, 11)
size is: 3.275739 Mb


{'body': '{}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

In [13]:
from inspect import getsource

In [15]:
print(getsource(lambdaDumpOnlineS3))

def lambdaDumpOnlineS3(event, *args):
  print(f'ecommece col list is {ECOMMERCE_COL_LIST}')
  # get all products from db
  df:pd.DataFrame = pd.DataFrame([i.data for i in ProductDatabase.scan()])
  # get online list from ECOMMERCE_COL_LIST
  onlineList:List[str] = yaml.load(requests.get(ECOMMERCE_COL_LIST).content)
  # filter df for item
  ## condition 1 master online is true
  condition1 = df['master_online'] == True
  ## condition 2 hema_name_en is not blank
  condition2 = df['hema_name_en'] != ''
  ## filtered df
  onlineDf:pd.DataFrame = df[condition1 & condition2].loc[:,onlineList]
  ### log shape and size
  print('shape is:',onlineDf.shape)
  print('size is:',sys.getsizeof(onlineDf.to_json(orient='split'))/1e6, 'Mb')

  # save file as gzip
  key = 'onlineData'
  bucket = INVENTORY_BUCKET_NAME
  path = '/tmp/inventory.json'
  ## export to gzip
  onlineDf.to_json(path ,orient='split',compression='gzip')
  ## upload file to s3
  S3.saveFile(key=key,path=path,bucket=bucket,
         

## get all products

In [18]:
df:pd.DataFrame = pd.DataFrame([i.data for i in ProductDatabase.scan()])
df.head()

,cprcode,iprcode,oprcode,ordertype,pr_abb,pr_active,pr_cgcode,pr_dpcode,pr_engname,pr_ggcode,...,preorder_delivery_type,preorder_fix_date,preorder_relative_day,meta_title,meta_keywords,meta_description,depth,priority_score,product_attribute_images,related_products
0,225407,225407,225407,Y,MOKU,True,87,21,KONJAC LINGUINI,010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,241101,241101,241101,Y,BUMILGOCHUJANG,True,06,08,BUMILGOCHUJANG,006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,190100,190100,190100,Y,WHITE SHIMEJI,True,05,19,WHITE SHIMEJI,143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,62644,62644,62644,Y,STUTE APRICOT JAM430,True,07,08,STUTE APRICOT JAM 430 G.,002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,192167,192167,192167,Y,**COLGATE TOTAL CHAR,True,10,09,**COLGATE TOTAL CHARCOAL DEEP CLEAN 150G,005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### filter products

In [31]:
condition1 = df['master_online'] == True
condition2 = df['hema_name_en'] != ''
onlineList:List[str] = yaml.load(requests.get(ECOMMERCE_COL_LIST).content)
onlineDf:pd.DataFrame = df[condition1 & condition2].loc[:,onlineList]
onlineDf.head()

<ipython-input-31-7921d4e71dc6>:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  onlineList:List[str] = yaml.load(requests.get(ECOMMERCE_COL_LIST).content)


,cprcode,iprcode,psqty,pr_country_en,online_category_l1_en,online_category_l2_en,online_category_l3_en,hema_brand_en,hema_sizedesc,hema_name_en,hema_name_th
0,225407,225407,1,Thailand,Delicatessen & Eggs,Ready Meals,Other Ready Meals,Moku,160g,Konjac Flat Noodle,บุกเส้นแบน ตราโมกุ
1,241101,241101,1,,Grocery,Cooking Sauces & Gravy,Soy & Asian Sauces,Bumil,250g,Gochujang,
3,62644,62644,1,,Grocery,Jam Honey & Spreads,Jams & Marmalades,Stute,430g,Apricot Jam,Apricot Jam
4,192167,192167,1,Thailand,Health & Beauty,Dental Care,Toothpaste,Colgate,150g,Total Charcoal Deep Clean,Total Charcoal Deep Clean
5,243619,243619,1,,Pet & Household,Toys,Dolls,Douglas,1 Pc,Amber Fox Stuffed Animal,


In [50]:
df.shape

(45459, 77)

In [43]:
df[condition1].shape ## filter for master_online

(20158, 77)

In [45]:
df[condition1 & condition2].shape ## filter products with missing hema_name

(20158, 77)

In [33]:
onlineDf[onlineDf.cprcode == 241490]

,cprcode,iprcode,psqty,pr_country_en,online_category_l1_en,online_category_l2_en,online_category_l3_en,hema_brand_en,hema_sizedesc,hema_name_en,hema_name_th
36320,241490,241490,1,,Pet & Household,Toys,Dolls,Douglas,1 Pc,Izzy Rainbow Owl Fuzzle Stuffed Animal,


In [49]:
url = 'https://product-bucket-dev-manual.s3-ap-southeast-1.amazonaws.com/onlineData'
r = pd.read_json(url, orient = 'split', compression = 'gzip')
r.shape

(20158, 11)

In [46]:
r.shape

(20158, 11)